In [1]:
import csv
import os
import time
import pandas as pd

from src.auto_acmg import AutoACMG, AutoACMGPrediction
from src.defs.genome_builds import GenomeRelease

In [2]:
# Get the current path
from src.core.config import settings

path_to_root = settings.PATH_TO_ROOT

In [6]:
# Extract criteria from the csv file
path = os.path.join(path_to_root, "tests", "assets", "e2e_variants", "comparison_criteria.csv")
print(f"Data path: {path}")
variants = []
with open(path, "rt") as inputf:
    reader = csv.DictReader(inputf)
    for record in reader:
        if record["variant"].startswith("#"):
            continue
        variants.append(
            (
                record["variant"],
                record["expected_criteria"].split(";"),
                record["comment"]
            )
        )

/home/gromdimon/Working/auto-acmg/tests/assets/e2e_variants/comparison_criteria.csv


In [7]:
variants[:5]

[('NM_176795.4(HRAS):c.173C>T',
  ['PS1', 'PS4', 'PM1', 'PM2', 'PM6', 'PP2', 'PP3'])]

In [77]:
def eval_autoacmg(pred, expected):
    crit_met = []
    for crit in pred.model_dump().values():
        if crit["prediction"] == AutoACMGPrediction.Positive:
            crit_met.append(crit["name"])
    tp = list(set(expected) & set(crit_met))
    fn = list(set(expected) - set(crit_met))
    fp = list(set(crit_met) - set(expected))
    return crit_met, tp, fn, fp

def eval_intervar(pred, expected):
    crit_met = []
    for crit in pred:
        if crit in ['Intervar', 'Build', 'Chromosome', 'Position', 'Ref_allele', 'Alt_allele', 'Gene']:
            continue
        if pred[crit] == 1:
            crit_met.append(crit)
    tp = list(set(expected) & set(crit_met))
    fn = list(set(expected) - set(crit_met))
    fp = list(set(crit_met) - set(expected))
    return crit_met, tp, fn, fp

In [65]:
import requests

def intervar_response(variant: str):
    """
    Implement searching for ACMG classification for SNVs and indels.
    Proxy requests to the `WinterVar <http://wintervar.wglab.org/>`_ backend.

    :param variant: request
    :return: ACMG classification
    :rtype: dict
    """
    auto_acmg = AutoACMG(variant, GenomeRelease.GRCh37)
    seqvar = auto_acmg.resolve_variant()
    chromosome = seqvar.chrom
    position = seqvar.pos
    reference = seqvar.delete
    alternative = seqvar.insert

    if not chromosome or not position or not reference or not alternative:
        return

    url = (
        f"http://wintervar.wglab.org/api_new.php?"
        f"queryType=position&chr={chromosome}&pos={position}"
        f"&ref={reference}&alt={alternative}&build=hg19"
    )
    backend_resp = requests.get(url)
    backend_resp.raise_for_status()
    return backend_resp.json()

In [ ]:
# Run AutoACMG predictions

# Create a pandas DataFrame
stats = pd.DataFrame(columns=[
    "Variant", "Expected Criteria",
    "AutoACMG Criteria", "AutoACMG Prediction time", "AutoACMG True Positives", "AutoACMG False Negatives", "AutoACMG False Positives", 
    "Intervar Criteria", "Intervar Prediction time", "Intervar True Positives", "Intervar False Negatives", "Intervar False Positives"
])

for var in variants:
    record = {
        "Variant": var[0], 
        "Expected Criteria": ";".join(var[1]),
        "Comment": var[2],
        "AutoACMG Criteria": "", 
        "AutoACMG Prediction time": 0, 
        "AutoACMG True Positives": "", 
        "AutoACMG False Negatives": "", 
        "AutoACMG False Positives": "", 
        "Intervar Criteria": "", 
        "Intervar Prediction time": 0, 
        "Intervar True Positives": "", 
        "Intervar False Negatives": "", 
        "Intervar False Positives": "",
        "AutoACMG Full Response": "",
        "Intervar Full Response": ""
    }
    
    # AutoACMG
    try:
        start_time = time.time()
        auto_acmg = AutoACMG(var[0], GenomeRelease.GRCh37)
        pred = auto_acmg.predict()
        end_time = time.time()
        # Evaluate the model
        crit_met, tp, fn, fp = eval_autoacmg(pred, var[1])
        record["AutoACMG Criteria"]: ";".join(crit_met)
        record["AutoACMG Prediction time"]: end_time - start_time
        record["AutoACMG True Positives"]: ";".join(tp)
        record["AutoACMG False Negatives"]: ";".join(fn)
        record["AutoACMG False Positives"]: ";".join(fp)
        record["AutoACMG Full Response"]: pred.model_dump()
    except Exception as e:
        print(f"Exception was raised for {var[0]} in AutoACMG:\n{e}")

    # Intervar
    try:
        start_time = time.time()
        resp = intervar_response(var[0])
        end_time = time.time()
        crit_met, tp, fn, fp = eval_intervar(resp, var[1])
        record["Intervar Criteria"]: ";".join(crit_met)
        record["Intervar Prediction time"]: end_time - start_time
        record["Intervar True Positives"]: ";".join(tp)
        record["Intervar False Negatives"]: ";".join(fn)
        record["Intervar False Positives"]: ";".join(fp)
        record["Intervar Full Response"]: resp
    except Exception as e:
        print(f"Exception was raised for {var[0]} in InterVar:\n{e}")

    stats.append(record, ignore_index=True)

output_path = os.path.join(path_to_root, "src", "jupyter", "stats.csv")
stats.to_csv(output_path, index=False)


2024-07-04 11:28:02.418 | DEBUG    | src.auto_acmg:__init__:70 - AutoACMG initialized with variant: NM_176795.4(HRAS):c.173C>T and genome release: GenomeRelease.GRCh37
2024-07-04 11:28:02.420 | INFO     | src.auto_acmg:predict:126 - Predicting ACMG criteria for variant: NM_176795.4(HRAS):c.173C>T
2024-07-04 11:28:02.420 | DEBUG    | src.auto_acmg:resolve_variant:89 - Resolving variant: NM_176795.4(HRAS):c.173C>T
2024-07-04 11:28:02.421 | DEBUG    | src.api.dotty:to_spdi:35 - GET request to: https://reev.cubi.bihealth.org/internal/proxy/dotty/api/v1/to-spdi?q=NM_176795.4(HRAS):c.173C>T&assembly=GRCh37
2024-07-04 11:28:02.524 | DEBUG    | src.auto_acmg:resolve_variant:93 - Resolved sequence variant: NM_176795.4(HRAS):c.173C>T
2024-07-04 11:28:02.526 | INFO     | src.auto_acmg:predict:133 - Classifying ACMG criteria for sequence variant NM_176795.4(HRAS):c.173C>T, genome release: GRCh37.
2024-07-04 11:28:02.529 | WARNING  | src.auto_acmg:predict:142 - Note, that PP5 and BP6 criteria are d